In [ ]:
#Importando pacotes que serão utilizados para a resolução do exercício
import numpy as np
import networkx as ntx
import matplotlib.pyplot as plt
import random
from networkx.algorithms import tree

In [ ]:
#função gera um grafo completo com arestas de caminhos aleatórios
def generate_random_graph(n, seed=10):
    random.seed(seed)

    G = ntx.expected_degree_graph(list(np.ones(n)*9), seed=None, selfloops=False)
    ntx.to_undirected(G)

    edges = ntx.edges(G)
    for edge in edges:
        ntx.set_edge_attributes(G,{edge:{"weight":random.randrange(1,10),"cmap":0.25}})
    return G

In [ ]:
#Função desenha um grafo
def draw_graph(G,modified = False, path_list = [], layout='circular', show_arrows = True, has_path = True,custom_edges=[]):
    f,ax = plt.subplots(figsize=(6,6))

    #Define um layout adequado
    if layout == 'circular':
        pos = ntx.circular_layout(G)
    else:
        pos = ntx.random_layout(G, seed=3)

    #Determina as posições de cada vértice
    x = []
    y = []

    for i in range(len(pos)):
        x.append(pos[i][0])
        y.append(pos[i][1])

    #Desenha as arestas de acordo com o problema
    if not modified:
        ntx.draw_networkx_edges(G, pos=pos, edge_color='dimgray',alpha=0.5)
    else:
        if has_path:
            custom_edges = []
            for i in range(len(path_list)-1):
                custom_edges.append((path_list[i],path_list[i+1]))
        edge_labels = ntx.get_edge_attributes(ntx.to_directed(G.subgraph(path_list)), "weight")
        del edge_labels[(path_list[0],path_list[-1])]
        del edge_labels[(path_list[-1],path_list[0])]
        if has_path:
            #Desenha um subgrafo direcionado dado um trajeto
            ntx.draw_networkx_edges(ntx.to_directed(G.subgraph(path_list)), pos=pos
    , edgelist = custom_edges, edge_color='r', label='weight', arrows = show_arrows)
            ntx.draw_networkx_edge_labels(ntx.to_directed(G.subgraph(path_list)), pos, edge_labels, rotate =False)
        else:
            ntx.draw_networkx_edges(G.subgraph(path_list), pos=pos
    , edgelist = custom_edges, edge_color='r', label='weight', arrows = show_arrows)

    #Plota cada vértice
    plt.scatter(x,y,edgecolor={'black'})

    #Adiciona o nome em cima de cada vértice
    eps=0.03
    for i, txt in enumerate(range(len(pos))):
        plt.annotate(txt, (x[i]+eps, y[i]+eps))
    plt.show()

In [ ]:
G = generate_random_graph(15)
draw_graph(G)

In [ ]:
v_ini = 0
v_final = 3

#Aplicando o método de dijkstra
min_path = ntx.dijkstra_path(G, v_ini, v_final, weight='weight')
min_distance = ntx.dijkstra_path_length(G,0,5,weight='weight')

print(f'Vértice inicial: {v_ini}, Vértice final: {v_final}')
print(f'Menor caminho: {min_path} \nMenor distância: {min_distance}')
draw_graph(G,modified = True, path_list = min_path)

In [ ]:
def prim(G):
  E = tree.minimum_spanning_edges(G, algorithm="prim", data=False)
  edges = list(E)
  sorted_edges = sorted(sorted(x) for x in edges)
  return sorted_edges

In [ ]:
G = generate_random_graph(n=26, seed=10)
E = prim(G)
draw_graph(G,layout='')


In [ ]:
#Aplicando o prim
draw_graph(G,modified = True, path_list = list(G.nodes), layout=' ', show_arrows = False, has_path = False,custom_edges=E)